In [1]:
import pandas as pd
import requests
import yaml

In [2]:
with open ('spotify_key.yml','r') as f:
    secrets = yaml.safe_load(f)

In [3]:
client_id = secrets['client_id']
client_secret = secrets['client_secret']

url = 'https://accounts.spotify.com/api/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

try:
        auth_req = requests.post(url, headers=headers, data=data)
        auth_req.raise_for_status()  
        print("Status Code", auth_req.status_code)
        print("JSON Response ", auth_req.json())
        
        access_token = auth_req.json().get('access_token')
        if access_token:
            headers = {'Authorization': f'Bearer {access_token}'}
        else:
            print("Access token not found in response.")
except requests.exceptions.RequestException as e:
        print("Error:", e)

Status Code 200
JSON Response  {'access_token': 'BQBhzK1k4pnAGMwdZxzyUq-Z6gOXMUYAxNh4bYgpN4I60Y-U4yeRO6jWx3GQDronOmPsd3MEKlcemEBKKbmjz91LqmDkV6yYVH_umWCJvSAoeywdJy0', 'token_type': 'Bearer', 'expires_in': 3600}


In [4]:
playlist_url = 'https://open.spotify.com/playlist/5zFVXcNZrCZsRmgtxJfVN9'

playlist_URI = playlist_url.split('/')[-1]
playlist_URI

playlist_endpoint = f'https://api.spotify.com/v1/playlists/{playlist_URI}/tracks'
offset = 0
playlist_lists = []

while True:
    api_response = requests.get(playlist_endpoint + f'?offset={offset}', headers=headers)
    
    playlist_total = api_response.json()['total']
    
    playlist_lists.append(api_response.json()['items'])

    offset += 100
    
    if offset > playlist_total:
        break

# track no.1 
print(playlist_lists[0][0])



{'added_at': '2024-02-18T17:17:14Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/31orreuzybil23lpzqqbxz5qlk44'}, 'href': 'https://api.spotify.com/v1/users/31orreuzybil23lpzqqbxz5qlk44', 'id': '31orreuzybil23lpzqqbxz5qlk44', 'type': 'user', 'uri': 'spotify:user:31orreuzybil23lpzqqbxz5qlk44'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6PUZZX4GCzeFS0GaDWxVwz'}, 'href': 'https://api.spotify.com/v1/artists/6PUZZX4GCzeFS0GaDWxVwz', 'id': '6PUZZX4GCzeFS0GaDWxVwz', 'name': 'Maher Zain', 'type': 'artist', 'uri': 'spotify:artist:6PUZZX4GCzeFS0GaDWxVwz'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK

In [5]:
api_response.text

'{\n  "href" : "https://api.spotify.com/v1/playlists/5zFVXcNZrCZsRmgtxJfVN9/tracks?offset=0&limit=100",\n  "items" : [ {\n    "added_at" : "2024-02-18T17:17:14Z",\n    "added_by" : {\n      "external_urls" : {\n        "spotify" : "https://open.spotify.com/user/31orreuzybil23lpzqqbxz5qlk44"\n      },\n      "href" : "https://api.spotify.com/v1/users/31orreuzybil23lpzqqbxz5qlk44",\n      "id" : "31orreuzybil23lpzqqbxz5qlk44",\n      "type" : "user",\n      "uri" : "spotify:user:31orreuzybil23lpzqqbxz5qlk44"\n    },\n    "is_local" : false,\n    "primary_color" : null,\n    "track" : {\n      "album" : {\n        "album_type" : "album",\n        "artists" : [ {\n          "external_urls" : {\n            "spotify" : "https://open.spotify.com/artist/6PUZZX4GCzeFS0GaDWxVwz"\n          },\n          "href" : "https://api.spotify.com/v1/artists/6PUZZX4GCzeFS0GaDWxVwz",\n          "id" : "6PUZZX4GCzeFS0GaDWxVwz",\n          "name" : "Maher Zain",\n          "type" : "artist",\n          "uri"

In [6]:
playlist_lists[0][0]['track']['uri']

'spotify:track:3OohydwTMW65eZKOrX5MPY'

In [7]:
df = pd.DataFrame()
page_index = 0

while page_index < len(playlist_lists):
    page = playlist_lists[page_index]
    df = pd.concat([df, pd.json_normalize(page, sep='_')])
    page_index += 1

print(df[['track_name', 'track_id']])


                                           track_name                track_id
0                             For the Rest of My Life  3OohydwTMW65eZKOrX5MPY
1   Sepanjang Hidup - For the Rest of My Life Mala...  2rn3iavjpkKfwxSeL7PwJC
2                           Ramadan - English Version  3AztiK5cDOBwEgumRMTFic
3                           Ramadan - English Version  3AztiK5cDOBwEgumRMTFic
4                                         Insha Allah  3zU9o0CFN2YVNlDspMTL5k
5                                   Peace Be Upon You  3OUA3YNLMblk5wWhp2b0VI
6                                   Number One For Me  6Y80VHVWtqIeNIbVYGJMcN
7                                             Mawlaya  6u2hOGlFVf1pxMsixyLkxb
8                                      My Little Girl  2cSsf8mBGMq3t2D1D8Chzs
9                                          Forgive Me  21wcTSiSlqpaCZjJPGWS5I
10                                     One Big Family  6tBqXgnhlyJq3QRfeLuvN5
11                                    Assalamu Alayka  21rXEU6sc

In [8]:
df.columns

Index(['added_at', 'is_local', 'primary_color',
       'added_by_external_urls_spotify', 'added_by_href', 'added_by_id',
       'added_by_type', 'added_by_uri', 'track_album_album_type',
       'track_album_artists', 'track_album_available_markets',
       'track_album_external_urls_spotify', 'track_album_href',
       'track_album_id', 'track_album_images', 'track_album_name',
       'track_album_release_date', 'track_album_release_date_precision',
       'track_album_total_tracks', 'track_album_type', 'track_album_uri',
       'track_artists', 'track_available_markets', 'track_disc_number',
       'track_duration_ms', 'track_episode', 'track_explicit',
       'track_external_ids_isrc', 'track_external_urls_spotify', 'track_href',
       'track_id', 'track_is_local', 'track_name', 'track_popularity',
       'track_preview_url', 'track_track', 'track_track_number', 'track_type',
       'track_uri', 'video_thumbnail_url'],
      dtype='object')

In [9]:
headers = {'Authorization': f'Bearer {access_token}'}

track_ids = df['track_id'].to_list()

start = 0
end = len(track_ids)
step = 50  # max 50 each time

api_track_info = []
api_track_feature_info = []

i = start
while i < end:
    x = i
    id_var = ','.join(track_ids[x:x + step])

    api_track_endpoint = f'https://api.spotify.com/v1/tracks?ids={id_var}'
    api_response_track = requests.get(api_track_endpoint, headers=headers)
    api_track_info.append(api_response_track.json())

    api_track_features_endpoint = f'https://api.spotify.com/v1/audio-features?ids={id_var}'
    api_response_feature = requests.get(api_track_features_endpoint, headers=headers)
    api_track_feature_info.append(api_response_feature.json())

    print(id_var)
    
    i += step


3OohydwTMW65eZKOrX5MPY,2rn3iavjpkKfwxSeL7PwJC,3AztiK5cDOBwEgumRMTFic,3AztiK5cDOBwEgumRMTFic,3zU9o0CFN2YVNlDspMTL5k,3OUA3YNLMblk5wWhp2b0VI,6Y80VHVWtqIeNIbVYGJMcN,6u2hOGlFVf1pxMsixyLkxb,2cSsf8mBGMq3t2D1D8Chzs,21wcTSiSlqpaCZjJPGWS5I,6tBqXgnhlyJq3QRfeLuvN5,21rXEU6scrVtABabUzLhYH,5GZ6liz6FVhzAx8VQKCSbe,0muVpp3jrFfELrSU1E6dhv,1TqMfcSxLOqQQ3pKnG2TPN,5hE7Z5sgSDZkod7et2QlQ2,11bBVZx06Ubs5uydDv54lV,4ScCgWoOzEI6wX2mC8Bjqj,0KcEMREqmRmjJQoDSNUWnT,0KcEMREqmRmjJQoDSNUWnT,66E2EffwlAijsCi5cl2tnP,3IrXGrtmW1H0yMhg17eAec,1jTRfreUeGih0W2maAXTdQ,0j3PlpT8tcLhk53bMvQIAJ,4RGro75nN8dg1SeXbP280r,3xG3xyjCzcjM64koC44cDw,2A5CWwyPNBUavyxX8suXHJ,0Vtihq3nwPI9mpCEFP5B5n,4GgCP6xlOePvrT2TVDITrm,20M6ixwrImHCFasLPGoHpC,1wDlg4jCxDGx50Ckp6JDtI,71ySvA5ks7fzJ9n7uujHVu,3TxhijKgvcrveiCGYd3eb3,66JY3pbNr7WoGmqNYOzToY,3TxhijKgvcrveiCGYd3eb3,13SzVFSO1u5KuXt67rTBEv,62pCVdXPkRBLhfo3e1DdaR,0kq1YQMzeTPSFP5zDVyNXn,7bfn9Mdm8KHjvzrMYSBibV,2RAN26cTDQJnM6N4hMNetq,6Osy7rFsI1cGQxSmRpuLw3,1VQI2F0o229lNoyFC9d6eh,3nV2jMu7JPCOJhhKvzQ6EY,2wsnjaEjtKi

In [10]:
type(api_track_info)

list

In [11]:
api_track_feature_info

[{'audio_features': [{'danceability': 0.606,
    'energy': 0.637,
    'key': 9,
    'loudness': -6.165,
    'mode': 0,
    'speechiness': 0.0406,
    'acousticness': 0.65,
    'instrumentalness': 0,
    'liveness': 0.167,
    'valence': 0.329,
    'tempo': 113.012,
    'type': 'audio_features',
    'id': '3OohydwTMW65eZKOrX5MPY',
    'uri': 'spotify:track:3OohydwTMW65eZKOrX5MPY',
    'track_href': 'https://api.spotify.com/v1/tracks/3OohydwTMW65eZKOrX5MPY',
    'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3OohydwTMW65eZKOrX5MPY',
    'duration_ms': 231987,
    'time_signature': 4},
   {'danceability': 0.624,
    'energy': 0.672,
    'key': 9,
    'loudness': -4.973,
    'mode': 0,
    'speechiness': 0.039,
    'acousticness': 0.716,
    'instrumentalness': 0,
    'liveness': 0.249,
    'valence': 0.409,
    'tempo': 112.633,
    'type': 'audio_features',
    'id': '2rn3iavjpkKfwxSeL7PwJC',
    'uri': 'spotify:track:2rn3iavjpkKfwxSeL7PwJC',
    'track_href': 'https://api.sp

In [12]:
api_track_info

[{'tracks': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6PUZZX4GCzeFS0GaDWxVwz'},
       'href': 'https://api.spotify.com/v1/artists/6PUZZX4GCzeFS0GaDWxVwz',
       'id': '6PUZZX4GCzeFS0GaDWxVwz',
       'name': 'Maher Zain',
       'type': 'artist',
       'uri': 'spotify:artist:6PUZZX4GCzeFS0GaDWxVwz'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',
      'PA',
      'PY',
      'PE',
      'PH',
      'PL',
      'PT',
      'SG',
      'SK',
      'ES',
      'SE',
      'CH',


In [14]:
dfs = [pd.json_normalize(item['tracks'], sep='_') for item in api_track_info]

df_concat_tracks = pd.concat(dfs)

print(df_concat_tracks.shape)

(47, 29)


In [15]:
df_concat_tracks.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album_id,album_images,album_name,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_uri,external_ids_isrc,external_urls_spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,231986,False,https://api.spotify.com/v1/tracks/3OohydwTMW65...,3OohydwTMW65eZKOrX5MPY,False,For the Rest of My Life,57,...,1Wul1gjulHQkgnZ1aZMgne,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Thank You Allah (Turkish Version),2009,year,14,album,spotify:album:1Wul1gjulHQkgnZ1aZMgne,USBBC1101483,https://open.spotify.com/track/3OohydwTMW65eZK...
1,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,235026,False,https://api.spotify.com/v1/tracks/2rn3iavjpkKf...,2rn3iavjpkKfwxSeL7PwJC,False,Sepanjang Hidup - For the Rest of My Life Mala...,50,...,4vCtNaJLGyqYRqKbjm9MKK,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Thank You Allah (Malay Version),2011-06-08,day,16,album,spotify:album:4vCtNaJLGyqYRqKbjm9MKK,USBBC1102073,https://open.spotify.com/track/2rn3iavjpkKfwxS...
2,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,248186,False,https://api.spotify.com/v1/tracks/3AztiK5cDOBw...,3AztiK5cDOBwEgumRMTFic,False,Ramadan - English Version,59,...,4U7712IFFZo2fqWFozK2rv,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Singles & Duets,2015-06-17,day,15,album,spotify:album:4U7712IFFZo2fqWFozK2rv,QM6MZ1512300,https://open.spotify.com/track/3AztiK5cDOBwEgu...
3,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,248186,False,https://api.spotify.com/v1/tracks/3AztiK5cDOBw...,3AztiK5cDOBwEgumRMTFic,False,Ramadan - English Version,59,...,4U7712IFFZo2fqWFozK2rv,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Singles & Duets,2015-06-17,day,15,album,spotify:album:4U7712IFFZo2fqWFozK2rv,QM6MZ1512300,https://open.spotify.com/track/3AztiK5cDOBwEgu...
4,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,268653,False,https://api.spotify.com/v1/tracks/3zU9o0CFN2YV...,3zU9o0CFN2YVNlDspMTL5k,False,Insha Allah,49,...,1Wul1gjulHQkgnZ1aZMgne,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Thank You Allah (Turkish Version),2009,year,14,album,spotify:album:1Wul1gjulHQkgnZ1aZMgne,USBBC1101488,https://open.spotify.com/track/3zU9o0CFN2YVNlD...


In [16]:
df_concat_tracks.columns

Index(['artists', 'available_markets', 'disc_number', 'duration_ms',
       'explicit', 'href', 'id', 'is_local', 'name', 'popularity',
       'preview_url', 'track_number', 'type', 'uri', 'album_album_type',
       'album_artists', 'album_available_markets',
       'album_external_urls_spotify', 'album_href', 'album_id', 'album_images',
       'album_name', 'album_release_date', 'album_release_date_precision',
       'album_total_tracks', 'album_type', 'album_uri', 'external_ids_isrc',
       'external_urls_spotify'],
      dtype='object')

In [17]:
dfs = [pd.json_normalize(item['audio_features'], sep='_') for item in api_track_feature_info]

df_concat_feature = pd.concat(dfs)

print(df_concat_feature.shape)

(47, 18)


In [18]:
df_concat_feature 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.606,0.637,9,-6.165,0,0.0406,0.6500,0.000000,0.1670,0.329,113.012,audio_features,3OohydwTMW65eZKOrX5MPY,spotify:track:3OohydwTMW65eZKOrX5MPY,https://api.spotify.com/v1/tracks/3OohydwTMW65...,https://api.spotify.com/v1/audio-analysis/3Ooh...,231987,4
1,0.624,0.672,9,-4.973,0,0.0390,0.7160,0.000000,0.2490,0.409,112.633,audio_features,2rn3iavjpkKfwxSeL7PwJC,spotify:track:2rn3iavjpkKfwxSeL7PwJC,https://api.spotify.com/v1/tracks/2rn3iavjpkKf...,https://api.spotify.com/v1/audio-analysis/2rn3...,235027,4
2,0.512,0.439,2,-8.228,0,0.0291,0.7070,0.000057,0.1050,0.254,102.882,audio_features,3AztiK5cDOBwEgumRMTFic,spotify:track:3AztiK5cDOBwEgumRMTFic,https://api.spotify.com/v1/tracks/3AztiK5cDOBw...,https://api.spotify.com/v1/audio-analysis/3Azt...,248187,4
3,0.512,0.439,2,-8.228,0,0.0291,0.7070,0.000057,0.1050,0.254,102.882,audio_features,3AztiK5cDOBwEgumRMTFic,spotify:track:3AztiK5cDOBwEgumRMTFic,https://api.spotify.com/v1/tracks/3AztiK5cDOBw...,https://api.spotify.com/v1/audio-analysis/3Azt...,248187,4
4,0.457,0.672,0,-4.280,0,0.0385,0.6220,0.000003,0.0861,0.400,84.761,audio_features,3zU9o0CFN2YVNlDspMTL5k,spotify:track:3zU9o0CFN2YVNlDspMTL5k,https://api.spotify.com/v1/tracks/3zU9o0CFN2YV...,https://api.spotify.com/v1/audio-analysis/3zU9...,268653,4
5,0.489,0.374,9,-7.093,1,0.0331,0.9110,0.000000,0.1380,0.498,114.165,audio_features,3OUA3YNLMblk5wWhp2b0VI,spotify:track:3OUA3YNLMblk5wWhp2b0VI,https://api.spotify.com/v1/tracks/3OUA3YNLMblk...,https://api.spotify.com/v1/audio-analysis/3OUA...,271397,4
6,0.633,0.721,10,-6.292,1,0.0329,0.0508,0.000000,0.3560,0.734,91.982,audio_features,6Y80VHVWtqIeNIbVYGJMcN,spotify:track:6Y80VHVWtqIeNIbVYGJMcN,https://api.spotify.com/v1/tracks/6Y80VHVWtqIe...,https://api.spotify.com/v1/audio-analysis/6Y80...,259560,4
7,0.670,0.723,11,-5.082,1,0.0308,0.2280,0.000000,0.0619,0.881,90.983,audio_features,6u2hOGlFVf1pxMsixyLkxb,spotify:track:6u2hOGlFVf1pxMsixyLkxb,https://api.spotify.com/v1/tracks/6u2hOGlFVf1p...,https://api.spotify.com/v1/audio-analysis/6u2h...,292067,4
8,0.507,0.612,0,-6.331,1,0.0344,0.1670,0.000000,0.1120,0.403,75.000,audio_features,2cSsf8mBGMq3t2D1D8Chzs,spotify:track:2cSsf8mBGMq3t2D1D8Chzs,https://api.spotify.com/v1/tracks/2cSsf8mBGMq3...,https://api.spotify.com/v1/audio-analysis/2cSs...,273120,4
9,0.633,0.597,6,-6.694,0,0.0290,0.1740,0.000000,0.2290,0.451,114.996,audio_features,21wcTSiSlqpaCZjJPGWS5I,spotify:track:21wcTSiSlqpaCZjJPGWS5I,https://api.spotify.com/v1/tracks/21wcTSiSlqpa...,https://api.spotify.com/v1/audio-analysis/21wc...,213507,4


In [19]:
df_concat_feature.keys()

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [20]:
df_mereged = df_concat_tracks.merge(df_concat_feature, on = 'id', how = 'left')
df_mereged.shape

(53, 46)

In [21]:
df_mereged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 46 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   artists                       53 non-null     object 
 1   available_markets             53 non-null     object 
 2   disc_number                   53 non-null     int64  
 3   duration_ms_x                 53 non-null     int64  
 4   explicit                      53 non-null     bool   
 5   href                          53 non-null     object 
 6   id                            53 non-null     object 
 7   is_local                      53 non-null     bool   
 8   name                          53 non-null     object 
 9   popularity                    53 non-null     int64  
 10  preview_url                   53 non-null     object 
 11  track_number                  53 non-null     int64  
 12  type_x                        53 non-null     object 
 13  uri_x  

In [22]:
df_mereged.to_csv('Alhuraibi_Alamoudi_Ahmed_Spotify_Playlist_Dataset.csv')